In [1]:
import sys, os, pathlib, re, configparser
from Levenshtein import distance

In [2]:
assets_dir = pathlib.Path.cwd().parent / "SeaPowerStreamingAssets"
assets_dir  = assets_dir.resolve()

vessels_dir = assets_dir / "vessels"
vessels_dir = vessels_dir.resolve()

vessel_dict = {}

for file in vessels_dir.iterdir():
    if(file.name.split('_', 1)[0] == 'all'):
        if 'all' not in vessel_dict:
            vessel_dict['all'] = {}
        vessel_name = file.name.split('_', 1)[-1].split('.')[0]
        vessel_dict['all'][vessel_name] = file.as_posix()
    elif(len(file.name.split('_', 2)) == 3):
        nation, type, vessel_name = file.name.split('_', 2)
        vessel_name = vessel_name.split('.')[0]
        if nation not in vessel_dict:
            vessel_dict[nation] = {}
        if type not in vessel_dict[nation]:
            vessel_dict[nation][type] = {}
        vessel_dict[nation][type][vessel_name] = file.as_posix()
    else:
        if 'misc' not in vessel_dict:
            vessel_dict['misc'] = []
        vessel_dict['misc'].append(file.as_posix())

In [3]:
ini_options = []

def gather_options(ini_path : pathlib.Path):
    option_blocks = {}
    setting_block = r"\[[A-Za-z0-9\_]+?\]"
    # section_index = 0
    last_block = ""
    last_sub_block = ""
    with open(ini_path, "r") as f:
        for line in f:
            if len(line) > 0:
                block_match = re.match(setting_block, line)
                if block_match:
                    if len(last_block) > 0 and block_match.group(0).strip().strip("[]")[:-1] == last_block[:-1]:
                        last_sub_block = block_match.group(0).strip().strip("[]")
                        option_blocks[last_block][last_sub_block] = {}
                    else:
                        last_block = block_match.group(0).strip().strip("[]")
                        option_blocks[last_block] = {}
    ini_options.append(option_blocks)

In [17]:
ini_variants = []
def gather_variants(ini_path : pathlib.Path) -> None:
    option_blocks = {"Variants" : {}, "General" : {}, "Default" : {}}
    setting_block = r"\[[A-Za-z0-9\_]+?\]"
    leading_comment = r"[\/\#]+?"
    last_block = ""
    idx = 0
    line_num = 0
    with open(ini_path, "r") as f:
        for line in f:
            if len(line) > 1:
                block_match = re.match(setting_block, line)
                comment_match = re.match(leading_comment, line)
                if block_match:
                    block_name = block_match.group(0).strip().strip("[]")
                    last_block = block_name
                    if "Variant" in block_name:
                        idx = block_name[-1]
                        option_blocks["Variants"][idx] = {}
                    else:
                        option_blocks[block_name] = {}
                else:
                    if not comment_match:
                        try:
                            option, value = line.split()[0].split("=")
                            if "Variant" in last_block:
                                option_blocks["Variants"][idx][option] = value
                            else:
                                option_blocks[last_block][option] = value
                        except Exception as e:
                            print(line_num, line)
                            print(ini_path)
                            raise e
            line_num += 1
    ini_variants.append(option_blocks)

In [19]:
ini_options = []
ini_variants = []
for nation_k, nation_v in vessel_dict.items():
    if isinstance(nation_v, dict):
        for type_k, type_v in vessel_dict[nation_k].items():
            if isinstance(type_v, dict):
                for vessel_k, vessel_v in vessel_dict[nation_k][type_k].items():
                    if 'variants' in vessel_v:
                        gather_variants(vessel_v)
                    else:
                        gather_options(vessel_v)
            elif isinstance(type_v, list):
                print("list: ", type_v)
                for vessel_name in vessel_dict[nation_k][type_k]:
                    if 'variants' in vessel_v:
                        gather_variants(vessel_v)
                    else:
                        gather_options(vessel_v)
    elif isinstance(nation_v, list):
        for vessel_path in nation_v:
            if 'variants' in vessel_v:
                gather_variants(vessel_v)
            else:
                gather_options(vessel_v)
len(ini_options)

190 [-Variant17]

/home/mortenh/.local/share/Steam/steamapps/common/Sea Power/Sea Power_Data/StreamingAssets/original/vessels/civ_ms_freighter_b_variants.ini


ValueError: not enough values to unpack (expected 2, got 1)

In [ ]:
for ini_content in ini_options:
    for k, v in ini_content.items():
        if len(v.keys()) > 0:
            print(k, ini_content[k])
    